In [38]:
# 用于获取最新的DTBS的A区中的 cz 纯债价值 pd 平底溢价率
# 由于原先的cz是每周获取一次，因此若需要每日更新一次则需要这个文件，考虑到wind数据容量有限选择增量更新
# cz这个数据感觉更新的相当晚，因此不要太早跑，安全起见5点之后甚至第二天跑

import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
with open(DTBS_path, 'rb') as f:
    DTBS = pickle.load(f)

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

In [39]:
# 找出cz为空的那些行
df = A_DF[A_DF['cz'].isnull()]

day_list = sorted(list(set(df['date'])))

need_query_mapping = {}

for day in day_list:
    need_query_mapping[day] = list(df[df['date']==day]['bond_code'])

need_query_mapping

{'2023-08-29': ['110088.SH',
  '113545.SH',
  '113631.SH',
  '123128.SZ',
  '123156.SZ',
  '127027.SZ',
  '127061.SZ',
  '110047.SH',
  '110063.SH',
  '110070.SH',
  '110087.SH',
  '110089.SH',
  '110092.SH',
  '111001.SH',
  '111002.SH',
  '111003.SH',
  '111007.SH',
  '111012.SH',
  '113025.SH',
  '113027.SH',
  '113046.SH',
  '113047.SH',
  '113058.SH',
  '113059.SH',
  '113064.SH',
  '113534.SH',
  '113535.SH',
  '113566.SH',
  '113591.SH',
  '113600.SH',
  '113615.SH',
  '113621.SH',
  '113623.SH',
  '113632.SH',
  '113636.SH',
  '113638.SH',
  '113640.SH',
  '113641.SH',
  '113649.SH',
  '113650.SH',
  '113653.SH',
  '113657.SH',
  '113663.SH',
  '118000.SH',
  '118005.SH',
  '118006.SH',
  '118008.SH',
  '118010.SH',
  '118018.SH',
  '118020.SH',
  '118022.SH',
  '118023.SH',
  '118028.SH',
  '118029.SH',
  '123012.SZ',
  '123013.SZ',
  '123031.SZ',
  '123046.SZ',
  '123052.SZ',
  '123076.SZ',
  '123078.SZ',
  '123085.SZ',
  '123091.SZ',
  '123106.SZ',
  '123118.SZ',
  '123124.S

In [40]:
from WindPy import w
w.start()

for day in day_list:
    
    bond_code_list = need_query_mapping[day]
    
    # 纯债价值列表
    bond_value_list = w.wsd(bond_code_list, "strbvalue", day, day, "").Data[0]
    
    print(bond_value_list)
    
    for bond_code_index in range(len(bond_code_list)):
        bond_code = bond_code_list[bond_code_index]
        bond_value = bond_value_list[bond_code_index]
        DTBS['A'][bond_code][day]['cz'] = bond_value
        DTBS['A'][bond_code][day]['pd'] = (DTBS['A'][bond_code][day]['csv'] / DTBS['A'][bond_code][day]['cz'] - 1) * 100

[96.0341, 106.7029, 100.6874, 99.5455, 82.0772, 104.1834, 94.2868, 111.5632, 107.7304, 106.877, 96.4825, 97.9513, 94.7867, 94.5013, 100.7647, 93.0174, 89.3103, 79.9365, 107.2441, 95.0335, 102.9048, 102.0786, 99.3564, 98.8025, 97.067, 103.3552, 106.4364, 101.7775, 103.0604, 99.574, 104.656, 102.4293, 103.3434, 101.1197, 94.232, 100.5616, 92.9042, 97.6413, 90.7903, 90.6356, 90.6931, 85.6318, 88.5107, 98.999, 89.1756, 85.4235, 90.7207, 78.7845, 77.3787, 86.6249, 95.8276, 89.1235, 88.3955, 75.4454, 101.5913, 103.3699, 97.7529, 101.3033, 101.626, 98.6232, 103.2203, 93.8592, 103.7583, 102.1446, 89.932, 81.653, 95.4617, 85.2545, 91.9261, 84.783, 95.9708, 89.0249, 81.1403, 88.9849, 85.7345, 107.9954, 116.876, 103.724, 103.8466, 99.2622, 100.6427, 100.9074, 103.4751, 104.3399, 101.941, 96.1464, 96.1634, 99.8933, 96.9438, 92.0676, 91.8755, 96.773, 84.901, 97.7851, 97.5715, 89.8407, 80.3507, 100.5321, 105.5133, 114.3361, 105.7704, 97.2467, 106.2986, 62.8465, 105.3424, 104.1547, 105.3748, 104.3019

In [44]:
DTBS['A']['113545.SH']['2023-08-29']

{'cpr': 40.8,
 'dp': 118.749,
 'dl': 159.54899999999998,
 'ytm': -2.23,
 'bl': 9.865,
 'trt': 2.49,
 'yl': 2.129,
 'csp': 9.96,
 'ia': 1,
 'xx': 0,
 'hs': 0,
 'csv': 84.33734939759036,
 'qs': 0,
 'qs15': 0,
 'qs30': 0,
 'xx15': 3,
 'xx30': 3,
 'cz': 106.7029,
 'pd': -20.960583641503316}

In [43]:
# 保存上面的这些信息的更改
f_save = open(DTBS_path, 'wb')
pickle.dump(DTBS, f_save)
f_save.close()